# TTT4135 - Assignment 1 - Martin Eggen & Martin Færevaag

## Problem 1 Information theory and data compression

This problem set addresses information theory and lossless data compression. The first
set of problems are theoretical only:

## 1a)
**Find the entropy for the following distributions:**
- $P(X=x) = 1/2$ for  $X\in \{0,1\}$
- $P(X=0)=0, P(X=x) = 1/2$ for  $X\in \{1,2\}$
- $P(X=0, Y=0) = 1/2, P(X = 0, Y = 1) = 1/4, P(X = 1, Y = 0) = 1/8, P(X = 1, Y = 1) = 1/8$
- $P(X=n)= 2^n, n\in\{1,2,3\ldots,\infty\}$

In [5]:
import numpy as np

#1a I
p = 1/2
H = -p*np.log2(p)-(1-p)*np.log2(1-p)
print("1a I) The entropy is", H)

# 1a II
p = np.array([1/2, 1/2]) #Ignore 0 because it is 0 chance
H = np.sum(-p*np.log2(p))
print("1a II) The entropy is", H)

#1a III
p = np.array([1/2,1/4,1/8,1/8])
H = np.sum(-p*np.log2(p))
print("1a III) The entropy is", H)

#1a IV
n = 1
run = True
ssum = 0
while run:
    verdi = -2**(-n)*np.log2(2**(-n))
    ssum = ssum + verdi
    n+=1
    if abs(verdi) <= 0.000001:
        run = False
print("1a IV) Then entropy is", round(ssum,2))

1a I) The entropy is 1.0
1a II) The entropy is 1.0
1a III) The entropy is 1.75
1a IV) Then entropy is 2.0


## 1b)
Let $X,Y \in \{0,1\}$. Let $P(Y=0 = 1/4, P(X=0|Y=0) = 1/2$ and $P(X=1|Y=1)=1/2$. Compute  $H(X), H(Y), H(X,Y), H(X|Y), H(Y|X)$ and $I(X;Y)$

In [6]:
P_Y = np.array([1/4, 1-1/4])
P_XgivenY = np.array([[1/2,1/2], # Rows: X=x, Columns Y=y
                        [1/2,1/2]])
P_X = P_XgivenY.dot(P_Y)

P_XandY = list()
P_XandY_log = list()
P_YgivenX = np.empty(shape=(2,2))
I_log = list()

for i in range(0,2):
    for j in range(0,2):
        P_XandY.append(P_X[i]*P_Y[j])
        P_XandY_log.append(P_X[j]*P_Y[i])
        temp = P_XgivenY[i,j]*P_Y[j]/P_X[i]
        P_YgivenX[i,j] = temp
        I_log.append(P_X[i]*P_Y[j]/(P_X[i]*P_Y[j]))
        
HX = -np.sum(P_X*np.log2(P_X))
HY = -np.sum(P_Y*np.log2(P_Y))
HXandY = -np.sum(np.array(P_XandY)*np.log2(np.array(P_XandY)))
HXgivenY = -np.sum(np.array(P_XandY)*np.log2(P_XgivenY.flatten()))
HYgivenX = -np.sum(np.array(P_XandY)*np.log2(P_YgivenX.flatten()))
IXY = HX - HXgivenY

print("H(X) = ",HX)
print("H(Y) = ",HY)
print("H(X,Y) = ",HXandY)
print("H(X|Y) = ",HXgivenY)
print("H(Y|X) = ",HYgivenX)
print("I(X;Y) = ",IXY)

H(X) =  1.0
H(Y) =  0.8112781244591328
H(X,Y) =  1.811278124459133
H(X|Y) =  1.0
H(Y|X) =  0.8112781244591329
I(X;Y) =  0.0


## 1c) 
(Hard) A Markov chain is a sequence $X1, X2, X3,\ldots$ of random variables that has the following property:

\begin{align}
P(Xn|X_{n-1}, X_{n-2}, X_{n-3}, \ldots) = P(Xn|Xn−1)
\end{align}
for all *n*. We can describe a Markov chain completely by its initial probabilities $P(X1)$ and
transition probabilities $P(Xn|Xn−1)$. Let $X ∈ \{0, 1\}$ and

\begin{align}
P(X = 0) = 1/2 \\
P(X_{n} = 0|X_{n-2} = 0) = 7/8 \\
P(X_{n} = 1|X_{n-1} = 1) = 7/8
\end{align}
Find the steady state distribution for X, which is the probability $P(Xn = x)$ for any *n*. Then
compute the entropy rate of the sequence $X1, X2, X3, \ldots, Xn$ as *n* goes to infinity.

In [7]:
def get_stationary_distribution(transition_matrix):
    eigenvalues, eigenvectors = np.linalg.eig(transition_matrix.T)
    stationary_distribution = eigenvectors[:, np.isclose(eigenvalues, 1)].flatten()
    return stationary_distribution/np.sum(stationary_distribution)

def get_entropy_rate(transition_matrix):
    steady_state = get_stationary_distribution(transition_matrix)
    entropy_rate = 0
    for i in range(0, len(steady_state)):
        for j in range(0, len(steady_state)):
            entropy_rate += steady_state[i]*transition_matrix[i,j]*np.log2(transition_matrix[i,j]) if transition_matrix[i,j] != 0 else 0
    return -entropy_rate

In [8]:
transition_matrix = np.array([[7/8,1/8],[1/8,7/8]])
steady_state_distribution = get_stationary_distribution(transition_matrix)

print("stady state distribution is: ",steady_state_distribution)
print("entropy rate is: ", get_entropy_rate(transition_matrix))

stady state distribution is:  [0.5 0.5]
entropy rate is:  0.5435644431995964


The next problems involve numerical exercises using Python. There are three data files
named _data[123].txt_ that shall be used for the problems below. They are all text files with a
single 0 or 1 per line. You can easily load the files with _numpy.loadtxt_ (see NumPy manual).

## 1d) 
Estimate the entropy for each of the three files.

In [9]:
def get_prob(data, max_val):
    symbols = dict()
    prob = list()
    for i in range(0, max_val+1):
        symbols[bin(i)[2:].zfill(len(data[0]))] = 0
    for i in data:
        symbols[i] += 1
    for key in symbols:
        prob.append(symbols[key]/len(data))
    return prob

In [10]:
data1 = np.loadtxt("data1.txt")
data1 = [str(int(i)) for i in data1]
data2 = np.loadtxt("data2.txt")
data2 = [str(int(i)) for i in data2]
data3 = np.loadtxt("data3.txt")
data3 = [str(int(i)) for i in data3]

datas = [data1, data2, data3]
P_datas = list()
for i in datas:
    prob = [i for i in get_prob(i,1) if i != 0] 
    P_datas.append(prob)

Hdatas = list()
for i in P_datas:
    Hdatas.append(-np.sum(i*np.log2(i)))

print("H(data1) = ",Hdatas[0])
print("H(data2) = ",Hdatas[1])
print("H(data3) = ",Hdatas[2])

H(data1) =  0.8113854260218433
H(data2) =  0.8112781244591328
H(data3) =  0.8107562272490967


## 1e) 
Estimate the entropy for each of the three files when considering two, three and four bits
at the time.

In [11]:
def merge_bits(file, num):
    new_array = list()
    for i in range(0, len(file), num):
        try:
            word = ""
            for j in range(0, num):
                word += str(int(file[i+j]))
            new_array.append(word)
        except:
            pass
    return new_array

In [12]:
datas_2bit = list()
for i in datas:
    datas_2bit.append(merge_bits(i,2))

P_datas_2bit = list()
for i in datas_2bit:
    prob = [i for i in get_prob(i,3) if i != 0] 
    P_datas_2bit.append(prob)

Hdatas_2bit = list()
for i in P_datas_2bit:
    Hdatas_2bit.append(-np.sum(i*np.log2(i))/2)

print("H(data1_2bit) = ",Hdatas_2bit[0])
print("H(data2_2bit) = ",Hdatas_2bit[1])
print("H(data3_2bit) = ",Hdatas_2bit[2])
##########################################################
datas_3bit = list()
for i in datas:
    datas_3bit.append(merge_bits(i,3))

P_datas_3bit = list()
for i in datas_3bit:
    prob = [i for i in get_prob(i,7) if i != 0] 
    P_datas_3bit.append(prob)

Hdatas_3bit = list()
for i in P_datas_3bit:
    Hdatas_3bit.append(-np.sum(i*np.log2(i))/3)

print("\nH(data1_3bit) = ",Hdatas_3bit[0])
print("H(data2_3bit) = ",Hdatas_3bit[1])
print("H(data3_3bit) = ",Hdatas_3bit[2])
##########################################################
datas_4bit = list()
for i in datas:
    datas_4bit.append(merge_bits(i,4))

P_datas_4bit = list()
for i in datas_4bit:
    prob = [i for i in get_prob(i,15) if i != 0] 
    P_datas_4bit.append(prob)

Hdatas_4bit = list()
for i in P_datas_4bit:
    Hdatas_4bit.append(-np.sum(i*np.log2(i))/4)

print("\nH(data1_4bit) = ",Hdatas_4bit[0])
print("H(data2_4bit) = ",Hdatas_4bit[1])
print("H(data3_4bit) = ",Hdatas_4bit[2])

H(data1_2bit) =  0.8113851861498752
H(data2_2bit) =  0.7499999027552386
H(data3_2bit) =  0.5672105369346512

H(data1_3bit) =  0.8113842324124635
H(data2_3bit) =  0.6666665437480547
H(data3_3bit) =  0.486741388652076

H(data1_4bit) =  0.8113824233775777
H(data2_4bit) =  0.24999990275523865
H(data3_4bit) =  0.4454849456126865


## 1f)
Attempt to estimate the entropy rate for each of the three files. Note that you will need
to approximate the rate by some finite symbols length n which is up to you to determine
(Note: The choice of n is important as increasing n arbitrarily will lead to a entropy rate
equal to zero. Why?).

In [13]:
N = list()

for i in range(5, 21):
    N.append(i)

for n in N:
    datas_Nbit = list()
    for i in datas:
        datas_Nbit.append(merge_bits(i,n))

    P_datas_Nbit = list()
    for i in datas_Nbit:
        prob = [i for i in get_prob(i,2**n-1) if i != 0] 
        P_datas_Nbit.append(prob)

    Hdatas_Nbit = list()
    for i in P_datas_Nbit:
        Hdatas_Nbit.append(-np.sum(i*np.log2(i)))

    print(f"{n} bit entropy rate of data1 = ",Hdatas_Nbit[0]/n)
    print(f"{n} bit entropy rate of data2 = ",Hdatas_Nbit[1]/n)
    print(f"{n} bit entropy rate of data3 = ",Hdatas_Nbit[2]/n)
    print("\nWhen increasing n arbitrariliy the entroyp rate goes to zero, becuase the number of possible states increases exponentially, but most of the states will have a probability of zero and will not contribute to the entropy, then you divide by n and the entropy rate goes to zero.")

5 bit entropy rate of data1 =  0.8113808989188748
5 bit entropy rate of data2 =  0.5561170065840938
5 bit entropy rate of data3 =  0.42135525722506956

When increasing n arbitrariliy the entroyp rate goes to zero, becuase the number of possible states increases exponentially, but most of the states will have a probability of zero and will not contribute to the entropy, then you divide by n and the entropy rate goes to zero.
6 bit entropy rate of data1 =  0.8113767725492664
6 bit entropy rate of data2 =  0.41666626572020776
6 bit entropy rate of data3 =  0.4051741490508978

When increasing n arbitrariliy the entroyp rate goes to zero, becuase the number of possible states increases exponentially, but most of the states will have a probability of zero and will not contribute to the entropy, then you divide by n and the entropy rate goes to zero.
7 bit entropy rate of data1 =  0.8113661111931163
7 bit entropy rate of data2 =  0.4999611873858086
7 bit entropy rate of data3 =  0.39360788078

## 1g)
Try and model each of the three data files as a Markov chain. Estimate the transition and
steady state parameters and compute the entropy. Compare with the entropy rate from
the previous problem. Which problem is likely to come from a binary Markov chain?

In [14]:
def get_transition_matrix(data):
    symbols = dict()
    max_val = 2**len(data[0])
    for i in range(0, max_val):
        symbols[bin(i)[2:].zfill(len(data[0]))] = dict()
        for j in range(0, max_val):
            symbols[bin(i)[2:].zfill(len(data[0]))][bin(j)[2:].zfill(len(data[0]))] = 0
    for i in range(0, len(data)-1):
        symbols[data[i]][data[i+1]] += 1
    transition_matrix = np.zeros((max_val,max_val))
    row = 0
    for i in symbols:
        column = 0
        count = sum(symbols[i].values())
        for j in symbols[i]:
            transition_matrix[row,column] = symbols[i][j]/count if count != 0 else 0
            column += 1
        row += 1
    return transition_matrix     

In [25]:
datas_transition_matrix = [get_transition_matrix(i) for i in datas]

print("transition matrix of data1 =\n",datas_transition_matrix[0])
print("\ntransition matrix of data2 =\n",datas_transition_matrix[1])
print("\ntransition matrix of data3 =\n",datas_transition_matrix[2])

print("\nsteady state of data1 =",get_stationary_distribution(datas_transition_matrix[0]))
print("\nsteady state of data2 =",get_stationary_distribution(datas_transition_matrix[1]))
print("\nsteady state of data3 =",get_stationary_distribution(datas_transition_matrix[2]))

print("\nentropy rate of data1 =",get_entropy_rate(datas_transition_matrix[0]))
print("\nentropy rate of data2 =",get_entropy_rate(datas_transition_matrix[1]))
print("\nentropy rate of data3 =",get_entropy_rate(datas_transition_matrix[2])) 

print("\nentropy rate of previous problem:\nentropy rate of data1 = 0.766563203997446820\nentropy rate of data2 = 0.2499948272960987320\nentropy rate of data3 = 0.34462187850422554\n\n")
print("wee see that the result for data1 and data3 are fairly close, but the result for data2 are very different, this could be due to sequence not being stationary")

print("\n\nI can't find much information about the issues exclusive to a binary markov chain. my guess would be that that they are limited in representing real world systems due to only having two states")

transition matrix of data1 =
 [[0.24975116 0.75024884]
 [0.25017335 0.74982665]]

transition matrix of data2 =
 [[0.         1.        ]
 [0.33333344 0.66666656]]

transition matrix of data3 =
 [[0.87448577 0.12551423]
 [0.04176471 0.95823529]]

steady state of data1 = [0.25006777 0.74993223]

steady state of data2 = [0.25000006 0.74999994]

steady state of data3 = [0.24967104 0.75032896]

entropy rate of data1 = 0.8113853918886793

entropy rate of data2 = 0.6887219002790084

entropy rate of data3 = 0.32389885074769115

entropy rate of previous problem:
entropy rate of data1 = 0.766563203997446820
entropy rate of data2 = 0.2499948272960987320
entropy rate of data3 = 0.34462187850422554


wee see that the result for data1 and data3 are fairly close, but the result for data2 are very different, this could be due to sequence not being stationary



A run length code can be defined as follows. Given a bit sequence
\begin{align}
\\
S = 111001111100111110000\\
\end{align}

we can code it as

\begin{align}
\\
C = 0325254\\
\end{align}

which can be interpreted as “zero 0s, three 1s, 2 0s, 5 1s, ...” etc. We have assumed that
we always start counting the zeros (here there are zero of them). We also need to determine
what the maximum length we can represent is, and how we will represent sequences longer
than this. Here we use the convention that if a sequence of eg. zeros are larger than some
maximum N, we write the code as N0M where M is the remainder of the symbols.

## 1h)
Implement a run-length code in Python and code the file _data3.txt_, using
\begin{align}
\\
N ∈ \{7, 15, 31, 63, 127\}\\
\end{align}

What is the coding gain for those three cases? How does it compare with the entropy
rate?

In [16]:
def run_length_encoder(data, n):
    bit_length = int(np.log2(n+1))
    bol = False
    encoded = ""
    count = 0
    i = 0
    while i < len(data):
        if count==n:
            encoded += bin(count)[2:].zfill(bit_length)
            count = 0
        elif data[i] != str(int(bol)):
            encoded += bin(count)[2:].zfill(bit_length)
            count = 0
            bol = not bol
        else:
            count += 1
            i += 1
    encoded += bin(count)[2:].zfill(bit_length)
    if count==n:
        encoded += bin(0)[2:].zfill(bit_length)
    return encoded

def run_length_decoder(data, n):
    bit_length = int(np.log2(n+1))
    bol = False
    decoded = ""
    for i in range(0, len(data), bit_length):
        decimal = int(data[i:i+bit_length],2)
        decoded += str(int(bol))*decimal
        if decimal != n:
            bol = not bol
    return decoded

In [17]:
def run_length_encoder_right_version(data, n):
    bit_length = int(np.log2(n+1))
    bol = False
    encoded = ""
    count = 0
    i = 0
    while i < len(data):
        if count==n:
            encoded += bin(count)[2:].zfill(bit_length)
            count = 0
            bol = not bol
        elif data[i] != str(int(bol)):
            encoded += bin(count)[2:].zfill(bit_length)
            count = 0
            bol = not bol
        else:
            count += 1
            i += 1
    encoded += bin(count)[2:].zfill(bit_length)
    return encoded

def run_length_decoder_right_version(data, n):
    bit_length = int(np.log2(n+1))
    bol = False
    decoded = ""
    for i in range(0, len(data), bit_length):
        decimal = int(data[i:i+bit_length],2)
        decoded += str(int(bol))*decimal
        bol = not bol
    return decoded

## 1i)
The file _data4.bin_ contains a version of the binary data from _data3.txt_ where the bits
have been packed into bytes and stored as a binary file. Use any compression program
of you choice (Zip, GZip, BZip2, 7Zip etc.) to compress the file and compare the results
with the entropy rate.

In [26]:
print("we initially misunderstood how the run length code should be implemented and thought that counts larger than n should be split into n and the rest immeditaletly after,") 
print("but this is not the case, the count should be n and then the next bit should be the opposite of the current bit, so we implemented the right version of the encoder,")
print("but this gave a worse result, so we decided to keep the wrong version of the encoder, but we kept the right version of the encoder as well\n")
N = [7, 15, 31, 63, 127]

for i in N:
    encoded_data3 = run_length_encoder(datas[2], i)
    print(f"\nN = {i} :")
    print(f"encoded data3 size = ",len(encoded_data3))
    print(f"encoded data3 gain = ",len(encoded_data3)/len(datas[2]))

encoded_data3 = run_length_encoder(datas[2], 63)

decoded_data3 = run_length_decoder(encoded_data3, 63)

for i in range(len(datas[2])):
    if datas[2][i] != decoded_data3[i]:
        print("error")
        break
else:
    print("no error")
    
print("\nBy using 7Zip, the file got compressed down to 196 kB = 1568000 bits")
print("That is a compression ratio of 196/516 KB = 0.38")

print("The best compression ratio was acheived by using N = 31")
print("Our code made it only 0.5% bigger than what 7Zip did")
print("our best compression here was rate was roughtly 0.376 while the entropy rate for data3 was 0.324")

we initially misunderstood how the run length code should be implemented and thought that counts larger than n should be split into n and the rest immeditaletly after,
but this is not the case, the count should be n and then the next bit should be the opposite of the current bit, so we implemented the right version of the encoder,
but this gave a worse result, so we decided to keep the wrong version of the encoder, but we kept the right version of the encoder as well


N = 7 :
encoded data3 size =  2260011
encoded data3 gain =  0.5388286113739014

N = 15 :
encoded data3 size =  1755900
encoded data3 gain =  0.4186391830444336

N = 31 :
encoded data3 size =  1576290
encoded data3 gain =  0.37581682205200195

N = 63 :
encoded data3 size =  1637850
encoded data3 gain =  0.39049386978149414

N = 127 :
encoded data3 size =  1844794
encoded data3 gain =  0.4398331642150879
no error

By using 7Zip, the file got compressed down to 196 kB = 1568000 bits
That is a compression ratio of 196/516 KB

In [28]:
N = [7, 15, 31, 63, 127]

for i in N:
    encoded_data3_right = run_length_encoder_right_version(datas[2], i)
    print(f"\nN = {i} :")
    print(f"encoded data3 size = ",len(encoded_data3_right))
    print(f"encoded data3 gain = ",len(encoded_data3_right)/len(datas[2]))

decoded_data3_right = run_length_decoder_right_version(encoded_data3_right, 127)

for i in range(len(datas[2])):
    if datas[2][i] != decoded_data3_right[i]:
        print("error")
        break
else:
    print("no error")

    
print("\nBy using 7Zip, the file got compressed down to 196 kB = 1568000 bits")
print("That is a compression ratio of 196/516 KB = 0.38")

print("The best compression ratio was acheived by using N = 63")
print("Our code made it only 0.5% bigger than what 7Zip did")
print("the lowest size we got with the right verion was roughly 1.69Mb, which is larger than the 1.576Mb we got with the wrong version")
print("our best compression rate here was roughtly 0.4 while the entropy rate for data3 was 0.324")


N = 7 :
encoded data3 size =  3559680
encoded data3 gain fast =  0.84869384765625

N = 15 :
encoded data3 size =  2386040
encoded data3 gain fast =  0.5688762664794922

N = 31 :
encoded data3 size =  1813040
encoded data3 gain fast =  0.4322624206542969

N = 63 :
encoded data3 size =  1693512
encoded data3 gain fast =  0.4037647247314453

N = 127 :
encoded data3 size =  1849106
encoded data3 gain fast =  0.44086122512817383
no error

By using 7Zip, the file got compressed down to 196 kB = 1568000 bits
That is a compression ratio of 196/516 KB = 0.38
The best compression ratio was acheived by using N = 31
Our code made it only 0.5% bigger than what 7Zip did


## Problem 2 Deep neural networks
This problem is designed to be very open and is aiming to be similar to the process of
training a deep neural network from scratch.\
\
You will find the file _problem2_template.py_ attached. This file contains a bare bones script
for training a very simple classifier on the **CIFAR 10** datasets (_https://www.cs.toronto.edu/
~kriz/cifar.html_). This is a classification task that has ten classes - **’plane’, ’car’, ’bird’, ’cat’,
’deer’, ’dog’, ’frog’, ’horse’, ’ship’ and ’truck’**.\
We want to you create a DNN classifier and use it to do image retrieval on the test set. For
example, if you decide to retrieve images of dogs, you should measure the accuracy, precision
and recall for this task. We also want you to use the AUPRC (area under the PR curve) to
rank the different systems you make.\
In your task you are free to add new layers, increase the number of nodes per layer, add
dropout and change the optimization algorithm if so be. The dataset is small, but the training
will take some time, so you will need to make sure you pick the lowest hanging fruits first.
The answer to this problem should of course be the precision-recall curve for the retrival
problem, but also a description of how you improved the original neural network with the
relevant results in tables and graphs.\
\
As a starting point you should to the following:
1. Run the script and make sure it works. You will need the two Python modules *torch*
and *torchvision*. Make a note of the time it takes to finish the training, and adjust the
number of epochs to make your turn-around between experiments quick enough.
2. Divide the training set into a training and validation set. You can do this manually or
using tools from *scikit-learn*.
3. Add layers to the neural network and observe the performance on the validation set.
4. Use *scikit-learn* to compute PR curves for the retrieval problem and compare the
results.

In [4]:
import numpy as np

import sklearn
from sklearn.model_selection import train_test_split

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Subset

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5),
                                                     (0.5, 0.5, 0.5))])
device = "cuda:0" if torch.cuda.is_available() else "cpu"
kwargs = {} if device=='cpu' else {'num_workers': 1, 'pin_memory': True}

batch_size=8

dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

# Define how much of the dataset should be validation set
val_size = 0.25

#Split dataset to create training- and validation set
train_idx, val_idx = train_test_split(list(range(len(dataset))),test_size = val_size)

trainset = Subset(dataset, train_idx) 
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, **kwargs)
valset   = Subset(dataset, val_idx) 
validloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=True, **kwargs)

# Import testset
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, **kwargs)

# Define classes
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 400)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x
    
net = Net()    # Define network
net.to(device) # Use CPU or GPU

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)


##########################################################################################################################  

epochs = 7
min_valid_loss = np.inf
for epoch in range(epochs):  # loop over the dataset multiple times

    training_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        # inputs, labels = data
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        # print statistics
        training_loss += loss.item()
        
    valid_loss = 0.0
    net.eval()     # Optional when not using Model Specific layer
    for data, labels in validloader:
        # Transfer Data to GPU if available
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()

        # Forward Pass
        target = net(data)
        # Find the Loss
        loss = criterion(target,labels)
        # Calculate Loss
        valid_loss += loss.item()
    
    print("\n\n")
    print(f"Epoch {epoch+1}")
    print(f"Training Loss: {(training_loss / len(trainloader)):.3}")
    print(f"Validation Loss: {(valid_loss / len(validloader)):.3}")
     
    if min_valid_loss > valid_loss:
        print(f"Validation Loss Decreased ({(min_valid_loss / len(validloader)):.2}--->{(valid_loss / len(validloader)):.3})")
        print("Saving The Model")
        min_valid_loss = valid_loss
         
        # Saving State Dict
        #torch.save(net.state_dict(), 'saved_model.pth')

    
##########################################################################################################################    
       
    
print("\nFinished Training")

correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%'
      % (100 * correct / total))

Files already downloaded and verified
Files already downloaded and verified



Epoch 1
Training Loss: 2.19
Validation Loss: 1.89
Validation Loss Decreased (inf--->1.89)
Saving The Model



Epoch 2
Training Loss: 1.66
Validation Loss: 1.48
Validation Loss Decreased (1.9--->1.48)
Saving The Model



Epoch 3
Training Loss: 1.42
Validation Loss: 1.34
Validation Loss Decreased (1.5--->1.34)
Saving The Model



Epoch 4
Training Loss: 1.29
Validation Loss: 1.34
Validation Loss Decreased (1.3--->1.34)
Saving The Model



Epoch 5
Training Loss: 1.19
Validation Loss: 1.2
Validation Loss Decreased (1.3--->1.2)
Saving The Model



Epoch 6
Training Loss: 1.1
Validation Loss: 1.17
Validation Loss Decreased (1.2--->1.17)
Saving The Model



Epoch 7
Training Loss: 1.02
Validation Loss: 1.2

Finished Training
Accuracy of the network on the 10000 test images: 58 %


In [10]:
#retrieve dogs

#outputs in the folling code returns an array containing the probability of each class. if the class for dog has the highest probability then we want to retrieve that image
#but i have no clue which index belongs to dog, cause i can't read the testloader file. there lets say index "2" is dog

for data in testloader:
    images, labels = data[0].to(device), data[1].to(device)
    outputs = net(images)
    prob = -10
    index = 0
    for i in range(outputs.numpy()):
        if outputs[i] > prob:
            prob = outputs[i]
            index = i
    if index == 2:
        print(images)

TypeError: only integer tensors of a single element can be converted to an index

In [ ]:
from sklearn.metrics import precision_recall_curve

# Test the model
print("Testing the model")
with torch.no_grad():
    predictions = []
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        predictions.append(outputs.cpu().numpy())

# Compute PR curve
predictions = np.concatenate(predictions, axis=0)
labels = testset.targets
precisions, recalls, thresholds = precision_recall_curve(labels, predictions[:, 1], pos_label=1)

# Plot PR curve
import matplotlib.pyplot as plt
plt.plot(recalls, precisions)
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.show()

# Notes

The first run took 90 seconds and had an accuracy of 60%\

After trying out different combinations of epochs, batch size, number of layers and different dropout functions, we only managed to get 61% accuracy. This takes 120 seconds to run.